# Tinh Chỉnh Tối Ưu Hóa Ưu Tiên Trực Tiếp (DPO)

Bài tập này sẽ hướng dẫn bạn quá trình tinh chỉnh mô hình ngôn ngữ bằng phương pháp DPO. Chúng ta sẽ sử dụng mô hình đã được huấn luyện SFT trước đó `SmolLM2-135M-Instruct`, nên chúng ta có thể thực hiện tinh chỉnh DPO ngay. Bạn cũng có thể sử dụng mô hình bạn đã huấn luyện trong [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb).

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
     <h2 style='margin: 0;color:blue'>Bài tập: Tinh chỉnh SmolLM2 với DPOTrainer</h2>
     <p>Lấy một tập dữ liệu từ Hugging Face hub và tinh chỉnh ưu tiên cho mô hình. </p> 
     <p><b>Các bài tập</b></p>
     <p>🐢 Sử dụng bộ dữ liệu `trl-lib/ultrafeedback_binarized`</p>
     <p>🐕 Thử nghiệm với bộ dữ liệu `argilla/ultrafeedback-binarized-preferences`</p>
     <p>🦁 Chọn bộ dữ liệu liên quan đến trường hợp thực tế bạn quan tâm, hoặc sử dụng mô hình bạn đã huấn luyện trong 
        <a href="../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb">1_instruction_tuning</a></p>
</div>

In [ ]:
# Cài đặt các thư viện cần thiết
# !pip install transformers datasets trl huggingface_hub

# Đăng nhập vào Hugging Face

from huggingface_hub import login

login()

# Để thuận tiện, bạn có thể tạo một biến môi trường chứa `token hub` của bạn dưới dạng HF_TOKEN

In [ ]:
# Các thư viện cần thiết

import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig

## Chuẩn bị dữ liệu

In [ ]:
# Tải tập dữ liệu

# TODO: 🦁🐕 thay đổi tập dữ liệu theo lựa chọn của bạn
dataset = load_dataset(path="trl-lib/ultrafeedback_binarized", split="train")

In [ ]:
# TODO: 🐕 Nếu bộ dữ liệu của bạn không được biểu diễn dưới dạng danh sách hội thoại
# bạn có thể sử dụng hàm `process_dataset` để chuyển đổi nó.

## Chọn mô hình

Chúng ta sẽ sử dụng mô hình `SmolLM2-135M-Instruct` đã được huấn luyện SFT, nên nó tương thích với DPO. Bạn cũng có thể sử dụng mô hình bạn đã huấn luyện trong [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb).

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; width:80%; color:black'>
     <p>🦁 thay đổi đường dẫn mô hình thành đường dẫn hoặc repo id của mô hình bạn đã huấn luyện trong <a href="../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb">1_instruction_tuning</a></p>
</div>


In [ ]:
# TODO: 🦁 thay đổi mô hình thành đường dẫn/repo id của mô hình bạn đã huấn luyện

model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Mô hình để tinh chỉnh
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.float16,
).to(device)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Đặt tên cho bản tinh chỉnh để lưu &/ tải lên
finetune_name = "SmolLM2-FT-DPO"
finetune_tags = ["smol-course", "module_2"]

## Tinh chỉnh mô hình với DPO

In [ ]:
# Các tham số huấn luyện
training_args = DPOConfig(
    # Kích thước batch huấn luyện trên mỗi GPU
    per_device_train_batch_size=4,
    # Số bước cập nhật để tích lũy trước khi thực hiện backward/update
    # Kích thước batch hiệu quả = per_device_train_batch_size * gradient_accumulation_steps
    gradient_accumulation_steps=4,
    # Tiết kiệm bộ nhớ bằng cách không lưu các activations trong lượt chuyển tiếp
    # Thay vào đó tính toán lại trong lượt chuyển ngược
    gradient_checkpointing=True,
    # Learning rate cơ bản
    learning_rate=5e-5,
    # Kế hoạch learning rate - 'cosine' giảm dần LR theo đường cong cosine
    lr_scheduler_type="cosine",
    # Tổng số bước huấn luyện
    max_steps=200,
    # Tắt lưu checkpoints trong quá trình huấn luyện
    save_strategy="no",
    # Tần suất ghi logs
    logging_steps=1,
    # Thư mục lưu kết quả
    output_dir="smol_dpo_output",
    # Số bước warmup learning rate
    warmup_steps=100,
    # Sử dụng bfloat16 để tăng tốc huấn luyện
    bf16=True,
    # Tắt logging wandb/tensorboard
    report_to="none",
    # Giữ lại tất cả các cột trong dataset kể cả không sử dụng
    remove_unused_columns=False,
    # Bật MPS cho thiết bị Mac
    use_mps_device=device == "mps",
    # ID mô hình cho tải lên HuggingFace Hub
    hub_model_id=finetune_name,
)

In [ ]:
trainer = DPOTrainer(
    # Mô hình cần huấn luyện
    model=model,
    # Cấu hình huấn luyện
    args=training_args,
    # Tập dữ liệu chứa các cặp phản hồi được ưu tiên/bị từ chối
    train_dataset=dataset,
    # Tokenizer để xử lý đầu vào
    processing_class=tokenizer,
    # Tham số temperature của DPO kiểm soát độ mạnh của mô hình ưu tiên
    # Giá trị thấp hơn (như 0.1) làm mô hình thận trọng hơn khi tuân theo ưu tiên
    beta=0.1,
    # Độ dài tối đa của chỉ thị đầu vào
    max_prompt_length=1024,
    # Độ dài tối đa kết hợp của chỉ thị + phản hồi
    max_length=1536,
)

In [ ]:
# Huấn luyện mô hình
trainer.train()

# Lưu mô hình
trainer.save_model(f"./{finetune_name}")

# Lưu lên huggingface hub nếu đã đăng nhập (HF_TOKEN đã được thiết lập)
if os.getenv("HF_TOKEN"):
    trainer.push_to_hub(tags=finetune_tags)

## 💐 Bạn đã hoàn thành!

Bài tập này đã hướng dẫn bạn từng bước cách tinh chỉnh mô hình `HuggingFaceTB/SmolLM2-135M` sử dụng `DPOTrainer`. Theo các bước này, bạn có thể điều chỉnh mô hình để thực hiện các tác vụ cụ thể một cách hiệu quả hơn. Nếu bạn muốn tiếp tục học khóa học này, đây là một số bước bạn có thể thử:

- Thử bài tập này ở các mức độ khó hơn
- Đánh giá PR của bạn học khác
- Cải thiện khóa học thông qua Issue hoặc PR